<a href="https://colab.research.google.com/github/SoniaPMi/Master-IA/blob/main/ML_Exercise02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import seaborn as sns
import pandas as pd
import plotly.express as px
from math import log

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
import plotly.express as px

from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import random 
random.seed(123)

In [ ]:
dataset = sns.load_dataset("penguins")
dataset.dropna(inplace=True)
dataset.head(10)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,Female
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,Male
12,Adelie,Torgersen,41.1,17.6,182.0,3200.0,Female
13,Adelie,Torgersen,38.6,21.2,191.0,3800.0,Male
14,Adelie,Torgersen,34.6,21.1,198.0,4400.0,Male


In [ ]:
dummies = pd.get_dummies(dataset.loc[:, ['species', 'island']])
numeric = dataset.select_dtypes('number')

X = pd.concat([dummies, numeric], axis=1)
Y = dataset.iloc[:,-1:]

X.shape, Y.shape

((333, 10), (333, 1))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

model = LogisticRegression(max_iter=300)
model.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



LogisticRegression(max_iter=300)

In [ ]:
model.classes_

array(['Female', 'Male'], dtype=object)

In [ ]:
accuracy_score(Y_test, model.predict(X_test))

0.8059701492537313

In [ ]:
accuracy_score(Y_train, model.predict(X_train))

0.849624060150376

In [ ]:
precision_score(Y_test,model.predict(X_test), average=None, labels=['Female','Male'])

array([0.85185185, 0.775     ])

In [ ]:
recall_score(Y_test,model.predict(X_test), average=None, labels=['Female','Male'])

array([0.71875   , 0.88571429])

In [ ]:
f1_score(Y_test,model.predict(X_test), average=None, labels=['Female','Male'])

array([0.77966102, 0.82666667])

In [ ]:
#Transform into probabilities
new_probs = []
for value in Y_test['sex']:
  if value == "Male":
    new_probs.append(1.0)
  else:
    new_probs.append(0.0)

#Transform into probabilities
new_probs_fm = []
for value in Y_test['sex']:
  if value == "Female":
    new_probs_fm.append(1.0)
  else:
    new_probs_fm.append(0.0)

In [ ]:
#model.predict_proba(X_test)[:,0] #female
#model.predict_proba(X_test)[:,1] #male

In [ ]:
#male
fpr, tpr, thresholds = roc_curve(new_probs, list(model.predict_proba(X_test)[:,1]), drop_intermediate=False)
df_roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr, 'threshold':thresholds})

In [ ]:
#female
fpr_fm, tpr_fm, thresholds_fm = roc_curve(new_probs_fm, list(model.predict_proba(X_test)[:,0]), drop_intermediate=False)
df_roc_fm = pd.DataFrame({'fpr':fpr_fm, 'tpr':tpr_fm, 'threshold':thresholds_fm})

In [ ]:
df_roc

,fpr,tpr,threshold
0,0.00000,0.000000,1.999626
1,0.00000,0.028571,0.999626
2,0.00000,0.057143,0.999102
3,0.00000,0.085714,0.999088
4,0.00000,0.114286,0.998031
...,...,...,...
63,0.87500,1.000000,0.029388
64,0.90625,1.000000,0.018483
65,0.93750,1.000000,0.009547
66,0.96875,1.000000,0.004178


In [ ]:
df_roc_fm

,fpr,tpr,threshold
0,0.000000,0.00000,1.997789
1,0.000000,0.03125,0.997789
2,0.000000,0.06250,0.995822
3,0.000000,0.09375,0.990453
4,0.000000,0.12500,0.981517
...,...,...,...
63,0.885714,1.00000,0.002988
64,0.914286,1.00000,0.001969
65,0.942857,1.00000,0.000912
66,0.971429,1.00000,0.000898


In [ ]:
#male
fig = px.line(df_roc, x='fpr', y='tpr', hover_data=['threshold'])
fig.show()

In [ ]:
#male
roc_auc_score(new_probs, list(model.predict_proba(X_test)[:,1]))

0.9044642857142857

In [ ]:
#female
fig = px.line(df_roc_fm, x='fpr', y='tpr', hover_data=['threshold'])
fig.show()

In [ ]:
roc_auc_score(new_probs_fm, list(model.predict_proba(X_test)[:,0]))

0.9044642857142857

In [ ]:
#ahora intentamos mejorar el modelo con un bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

base_estimator = LogisticRegression(max_iter=300)
model = BaggingClassifier(base_estimator, n_estimators=12, max_samples=10, max_features=7)

In [ ]:
model.fit(X_train, Y_train)
print(model.score(X_test, Y_test))
base_estimator.fit(X_train, Y_train)
print(base_estimator.score(X_test, Y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.8656716417910447
0.8059701492537313


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [ ]:
#RandomForest
